用于转化图像类型和通道

In [ ]:
import cv2
import numpy

img_path = "data//train//Image1_1.jpg"
img = cv2.imread(img_path)
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_array = numpy.array(img_RGB)

print(img_array.shape) 
writer.add_image("test", img_array, 3, dataformats='HWC')
writer.close()

用于creat Dataset

In [ ]:
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from dataset_define import *


if __name__ == '__main__':

    #1. create my dataset
    transform = transforms.Compose([transforms.ToTensor()])        #opt:transforms.Grayscale(num_output_channels=1)，  transforms.resize(32)
    root_dir = "data"
    dataset_type = "train"
    label = "RGB"
    RGB_dataset = MyData(root_dir, label, dataset_type, transform)
    
    # #output---->注意data的输出格式应为[batch_size,C,H,W]
    # train_loader = DataLoader(RGB_dataset, batch_size = 16)      
    # writer = SummaryWriter("attempt_1_logs")
    # step = 0
    # for data in train_loader:
    #     writer.add_images('RGBs', data, step)         #data.Size([6, 211, 3, 32, 32])
    #     step = step + 1
    # writer.close()

将整数标签转为one-hot编码


In [3]:
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from dataset_define import *
import torch

In [5]:
patch_num = 211
tensor = torch.tensor(range(patch_num))

N = tensor.size(0)
num_classes = patch_num
one_hot = torch.zeros(N, num_classes).long()
a = one_hot.scatter_(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(N, num_classes).long())